<a href="https://colab.research.google.com/github/djili/data-processing/blob/main/log.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Log Analytics
Cet exercice consiste à lire un fichier(lien) de logs Apache public, à effectuer une série de transformations et d'actions. Les transformations devront utiliser des opérations comme map, filter, flatMap, union, intersection, groupByKey, et reduceByKey.

1.   Lire un fichier de logs Apache;
2.   Extraire les adresses IP et les URI demandés;
3.   Filtrer les requêtes pour obtenir uniquement celles avec des codes de statut 200;
4.   Compter le nombre de requêtes par adresse IP;
5.   Trouver les URI les plus fréquemment demandés;
6.   Identifier les adresses IP uniques ayant accédé au serveur;


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_extract, count, desc
import pyspark.sql.functions as F

In [2]:
# Démarrer une session Spark
spark = SparkSession.builder.appName("Log analytics").getOrCreate()
spark

In [3]:
# Lire le fichier le fichier text
txt = spark.read.text("/content/log.txt")
txt.show()

+--------------------+
|               value|
+--------------------+
|83.149.9.216 - - ...|
|83.149.9.216 - - ...|
|83.149.9.216 - - ...|
|83.149.9.216 - - ...|
|83.149.9.216 - - ...|
|83.149.9.216 - - ...|
|83.149.9.216 - - ...|
|83.149.9.216 - - ...|
|83.149.9.216 - - ...|
|83.149.9.216 - - ...|
|83.149.9.216 - - ...|
|83.149.9.216 - - ...|
|83.149.9.216 - - ...|
|83.149.9.216 - - ...|
|83.149.9.216 - - ...|
|83.149.9.216 - - ...|
|83.149.9.216 - - ...|
|83.149.9.216 - - ...|
|83.149.9.216 - - ...|
|83.149.9.216 - - ...|
+--------------------+
only showing top 20 rows



In [4]:
# Définition du pattern pour parser les logs Apache
LOG_PATTERN = r'^(\S+) - - \[([^\]]+)\] "(\S+) (\S+) (\S+)" (\d+) (\d+) "([^"]*)" "([^"]*)"'

# Extraction des champs avec regexp_extract
parsed_logs = txt.select(
    regexp_extract('value', LOG_PATTERN, 1).alias('ip'),
    regexp_extract('value', LOG_PATTERN, 4).alias('uri'),
    regexp_extract('value', LOG_PATTERN, 6).cast('integer').alias('status')
).filter(col('ip') != "")  # Supprime les lignes non parsées

In [5]:
# Filtrage des requêtes avec code de statut 200
successful_requests = parsed_logs.filter(col("status") == 200)

In [6]:
# Nombre de requêtes par adresse IP
requests_per_ip = successful_requests.groupBy("ip").agg(
    count("*").alias("request_count")
)
print("Nombre de requêtes par adresse IP (top 10):")
requests_per_ip.orderBy(desc("request_count")).show(10, truncate=False)


Nombre de requêtes par adresse IP (top 10):
+---------------+-------------+
|ip             |request_count|
+---------------+-------------+
|66.249.73.135  |419          |
|46.105.14.53   |364          |
|130.237.218.86 |288          |
|50.16.19.13    |113          |
|209.85.238.199 |102          |
|75.97.9.59     |93           |
|68.180.224.225 |91           |
|198.46.149.143 |82           |
|208.115.111.72 |72           |
|108.171.116.194|65           |
+---------------+-------------+
only showing top 10 rows



In [7]:
# URI les plus fréquemment demandés
popular_uris = successful_requests.groupBy("uri").agg(
    count("*").alias("request_count")
)
print("\nURI les plus fréquemment demandés (top 10):")
popular_uris.orderBy(desc("request_count")).show(10, truncate=False)


URI les plus fréquemment demandés (top 10):
+-------------------------------+-------------+
|uri                            |request_count|
+-------------------------------+-------------+
|/favicon.ico                   |788          |
|/style2.css                    |532          |
|/reset.css                     |528          |
|/images/jordan-80.png          |519          |
|/images/web/2009/banner.png    |506          |
|/blog/tags/puppet?flav=rss20   |488          |
|/projects/xdotool/             |220          |
|/?flav=rss20                   |217          |
|/                              |194          |
|/projects/xdotool/xdotool.xhtml|147          |
+-------------------------------+-------------+
only showing top 10 rows



In [8]:
# Adresses IP uniques ayant accédé au serveur
unique_ips = successful_requests.select("ip").distinct()
print("\nAdresses IP uniques ayant accédé au serveur (total):")
print(unique_ips.count())


Adresses IP uniques ayant accédé au serveur (total):
1615


In [10]:
spark.stop()
